- haal alle resultaten op uit fd_stats_resultaten_view, met geconformeerde teamnamen, en join met wedstrijden voor wedstrijdsleutel
- Join met tabel f_resultaten en check of wedstrijd + datum al voorkomt.
- Zo niet INSERT
- Update de updatetabel

In [10]:
import requests
import pandas as pd
import sqlalchemy
import psycopg2
import datetime

# connection variables
URL = 'football-fortunes.cj5o1qrvsyia.eu-west-1.rds.amazonaws.com'
database_name = 'postgres'
port = 5432
username = 'master'
ww = 'wachtwoord'

# Create SQL Alchemy database connection
db_url = f"postgresql://{username}:{ww}@{URL}:{port}/{database_name}"
engine = sqlalchemy.create_engine(db_url)
sqal_connection = engine.connect()

# Create Psychop connection
psy_connection = psycopg2.connect(
    host = URL,
    port = port,
    database = database_name,
    user= username,
    password = ww
)
psy_cursor = psy_connection.cursor()

# Haal alle wedstrijden op uit fd_stats_resultaten_VIEW, waarbij teamnamen geconformeerd en datum is DATE
# JOIN met d_wedstrijden om wedstrijdsleutel op te halen
# JOIN met f_resultaten om vast te stellen welke wedstrijden nog niet verwerkt zijn
# INSERT betreffende wedstijden
psy_cursor.execute("""
INSERT INTO dwa.f_resultaten  
SELECT
    r.s_wedstrijd			 
    ,r.datum				
    ,cast(r.fthg as Decimal)
    ,cast(r.ftag as Decimal)
    ,r.ftr
    ,cast(r.hthg as Decimal)
    ,cast(r.htag as Decimal)
    ,r.htr
    ,cast(r.hsh as Decimal)
    ,cast(r.ash as Decimal)
    ,cast(r.hst as Decimal)
    ,cast(r.ast as Decimal)
    ,cast(r.hf as Decimal)
    ,cast(r.af as Decimal)
    ,cast(r.hc as Decimal)
    ,cast(r.ac as Decimal)
    ,cast(r.hy as Decimal)
    ,cast(r.ay as Decimal)
    ,cast(r.hr as Decimal)
    ,cast(r.ar as Decimal)
FROM (
    SELECT 
        dw.s_wedstrijd, datum
        ,fthg, ftag, ftr, hthg, htag, htr, hsh, ash
        ,hst, ast, hf, af, hc, ac, hy, ay, hr, ar
    FROM dsa.fd_stats_resultaten_view fdr
    LEFT JOIN dwa.d_wedstrijden dw 
        on (fdr.hometeam = dw.hometeam and fdr.awayteam = dw.awayteam)
    ORDER BY div, datum
    ) r
LEFT JOIN dwa.f_resultaten fr 
    ON r.s_wedstrijd = fr.s_wedstrijd AND r.datum = fr.date
WHERE fr.s_wedstrijd IS NULL
;     

UPDATE dsa.update
SET update_datum = current_date
WHERE tabelnaam = 'f_resultaten'
;                      
""")  

# Commit de transactie en sluit de verbinding
sqal_connection.commit()
psy_connection.commit()
psy_cursor.close()
sqal_connection.close()
psy_connection.close() 